# Titanic. A learning journey

## Introduction
This is a mix that I'm learning from different Titanic noteboks. The main inspiratiors are

* [Titanic Survival Predictions (Beginner)](https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner)
* [A Data Science Framework: To Achieve 99% Accuracy](https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy)

Contents: Import Necessary Libraries Read In and Explore the Data Data Analysis Data Visualization Cleaning Data Choosing the Best Model Creating Submission File Any and all feedback is welcome!

## Table of Contents
1. [Define the problem](#ch1)
1. [Read the data](#ch2)
1. [Discover correlations](#ch3)
1. [Guess the missing ages](#ch4)

<a id="ch1"></a>

## 1. Define the problem

First thing first. What we are trying to do? 

Titanic. Belfast 1912. The largest ship ever made was struck by an iceberg and sank in his maiden vollage. Safety deficencies caused that more tan 1,500 of the 2,224. Our task is guess if a passenger will survive or not.

Well, let's do it 

<a id="ch2"></a>


## 2. Read the data

To start we begin reading and exploring the data 


In [ ]:
#data analysis libraries 
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#import train and test CSV files
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

#take a look at the training data
print (train.info())
train.describe(include="all")

# A small utility to see correlation
def ShowCorrelation (property, ax = None):
    if ax is None:
        fig, ax = plt.subplots(1, 1,figsize=(8,6))

    sns.pointplot(x=property, y=SURVIVED, data=train, ax = ax )
    print (train[[SURVIVED,property]].groupby(property, as_index=False).agg(['count', 'mean']))


In [ ]:
# A quick lock to the content
train.head()

In [ ]:
# Where there are missing values
missingTrain = pd.isnull(train).sum()
print (missingTrain[missingTrain>0])
missingTest = pd.isnull(test).sum()
print (missingTest[missingTrain>0])


### Data recap

   Well, after doig a first look to the data. What we can say?

   * PassengerId: Id of the passenger, doesn't seems relevant
   * Survived: What we want to know (0/1)
   * Pclass: A proxy for SES(Socio-Economic status) (1/2/3). 
   * Name: String with the name. Could we extract some information from here? 
   * Sex: Descriptior of the sex (male/female)
   * Age: Age of the person. It seemsimportat but we have a lot of nulls. We have to found an strategy assign this number
   * SibSp: Number of siblings spouses abroad  (0- 8)
   * Parch: Number of parent children abroad (0 - 9)
   * Ticket: String Id of the ticket
   * Fare: Passenger Fare
   * Cabin: Cabin used. A lot of nulls
   * Embark: Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton). There are 2 for 891 without this data. We can assign to the more used
  
      

   First thoughts

   * PClass is a integer but could make sense to tranform to a category?
   * Name + SibSp + Parch + ticket + cabin. Could be used to identify families? 

<a id="ch3"></a>
## Discover correlations
  
    First we will try to discover correlations between values and result



In [ ]:
SURVIVED = "Survived"
for x in train:
    if ((x != SURVIVED) and (train[x].unique().size <=10)):
        print (x, ": ", train[x].unique().size)
        print (train[[SURVIVED,x]].groupby(x, as_index=False).agg(['count', 'mean']))


In [ ]:
#draw a bar plot of survival by sex
SEX = "Sex";
PCLASS = "Pclass";
EMBARKED = "Embarked";
SIBSP = "SibSp";
PARCH = "Parch";
AGE = "Age";
fig, saxis = plt.subplots(2, 3,figsize=(16,12))
sns.barplot(x=SEX, y=SURVIVED, data=train, ax =saxis[0,0])
sns.barplot(x=PCLASS, y=SURVIVED, data=train, ax =saxis[0,1])
sns.barplot(x=EMBARKED, y=SURVIVED, data=train,order=['S','C','Q'], ax=saxis[0,2])
sns.pointplot(x=SIBSP , y=SURVIVED,  data=train, ax = saxis[1,0])
sns.pointplot(x=PARCH , y=SURVIVED,  data=train, ax = saxis[1,1])
sns.pointplot(x=AGE , y=SURVIVED,  data=train, ax = saxis[1,2])

Well, this is interesting for what it says and for what is missig.
Class, sex are very important
also it seems that there are a correlation between size of the family (Parch, sibSp) and probabilities of supervivence
what is missing is the age, as is provided is not very useful. We will arrange the age as it is in the notebook of LD Freeman, and by the way, it seems that he is also doing a interesting thing as is extract the Title form the name. Ok, let's do it 

<a id="ch4"></a>
##Guess the missing ages

In [ ]:
# what we have for title is this
train['Title'] = train['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
test['Title'] = train['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]

# as the titles seems related with the age, we can use to fill the age that is missing

AverageAgeByTitle = train[['Title',AGE]].groupby('Title', as_index=False).mean();
AverageAgeByTitle.reset_index(inplace=True);
print(AverageAgeByTitle);

In [ ]:
# guess the age 
PASSENGER_ID= "PassengerId"
TITLE = "Title"
df = train[[PASSENGER_ID, TITLE, AGE]].merge(AverageAgeByTitle, on=TITLE, how='left');
train[AGE][train[AGE].isnull()]=df['Age_y']

df = test[[PASSENGER_ID, TITLE, AGE]].merge(AverageAgeByTitle, on=TITLE, how='left');
test[AGE][test[AGE].isnull()]=df['Age_y']


In [ ]:

#Survived: What we want to know (0/1)
#Pclass: A proxy for SES(Socio-Economic status) (1/2/3).
#Name: String with the name. Could we extract some information from here?
#Sex: Descriptior of the sex (male/female)
#Age: Age of the person
#SibSp: Number of siblings spouses abroad (0- 8)
#Parch: Number of parent children abroad (0 - 9)
#Ticket: String Id of the ticket
#Fare: Passenger Fare
#Cabin: Cabin used. A lot of nulls
#Embark: Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

from sklearn.preprocessing import OneHotEncoder


NAME = "Name"
#train.drop([PASSENGER_ID, NAME]), axis=1, inplace=True);
#test.drop([PASSENGER_ID, NAME], axis=1, inplace=True);

# Fill NA
train[EMBARKED].fillna(train[EMBARKED].mode()[0], inplace = True)
test[EMBARKED].fillna(test[EMBARKED].mode()[0], inplace = True)

# One Hot Encoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
categories = [SEX, EMBARKED]

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[categories]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test[categories]))

# Remove categorical columns (will replace with one-hot encoding)
train.drop(categories, axis=1)
test.drop(categories, axis=1)

# Add one-hot encoded columns to numerical features
train = pd.concat([train, OH_cols_train], axis=1)
test = pd.concat([test, OH_cols_test], axis=1)


In [ ]:

AGEBIN = 'AgeBin'
#Age
train[AGEBIN] = pd.cut(train[AGE].astype(int), 5)
test[AGEBIN] = pd.cut(test[AGE].astype(int), 5)
ShowCorrelation (AGEBIN);

FAMILY_SIZE = 'FamilySize'
train[FAMILY_SIZE] = train[SIBSP] + train[PARCH]
test[FAMILY_SIZE] = train[SIBSP] + train[PARCH]

#OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

#label = LabelEncoder()
#train[SEX_CODE] = label.fit_transform(train[SEX])

#for dataset in data_cleaner:    
#    dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])

#for dataset in data_cleaner:    
#    #Discrete variables
#    dataset['FamilySize'] = dataset ['SibSp'] + dataset['Parch'] + 1
#
#    dataset['IsAlone'] = 1 #initialize to yes/1 is alone
#    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # now update to no/0 if family size is greater than 1

    #quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
#    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]


    #Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
#    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #Age Bins/Buckets using cut or value bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
#    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)


    
#cleanup rare title names
#print(data1['Title'].value_counts())
#stat_min = 10 #while small is arbitrary, we'll use the common minimum in statistics: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
#title_names = (data1['Title'].value_counts() < stat_min) #this will create a true false series with title name as index

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
#data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
#print(data1['Title'].value_counts())
#print("-"*10)


#preview data again
#data1.info()
#data_val.info()
#data1.sample(10)

In [ ]:
fig, saxis = plt.subplots(1, 3,figsize=(16,12))
ShowCorrelation (SIBSP, saxis[0]);
ShowCorrelation (PARCH, saxis[1]);
ShowCorrelation (FAMILY_SIZE, saxis[2]);

In [ ]:
# 1. Create the transformer steps and bundle it
#1.a Guessing the age from the Title extracted for the name
#1.b Assigning the nulls in the embarked port to the mode
#1.c Group the age in five groups
#1.d. Hot encode sex and port of embark

# Pending --- 
# Create familiSize joining SibSp and arch

# To analize
# Is good to create an isalone field?S
# Makes sense to create a faresBin?

# Preprocessing for numerical data
#numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
#categorical_transformer = Pipeline(steps=[
#    ('imputer', SimpleImputer(strategy='most_frequent')),
#    ('onehot', OneHotEncoder(handle_unknown='ignore'))
#])

# Bundle preprocessing for numerical and categorical data
#preprocessor = ColumnTransformer(
#    transformers=[
#        ('num', numerical_transformer, numerical_cols),
#        ('cat', categorical_transformer, categorical_cols)
#    ])


# 2. Define the model

#    model = RandomForestRegressor(n_estimators=100, random_state=0)

# 3. Setup model and transformers in the Pipeline

#from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
#my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                              ('model', model)
#                             ])

# Preprocessing of training data, fit model 
#my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
#preds = my_pipeline.predict(X_valid)

# Evaluate the model
#score = mean_absolute_error(y_valid, preds)
#print('MAE:', score)